## Import the required libraries.

In [1]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
import math
import copy

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## Helper functions.

In [2]:
###############################################################################
# HELPER FUNCTIONS
###############################################################################

def homogeneous(ROLL, PITCH, YAW, X, Y, Z):
    T = np.eye(4,4);
    T[0,0] = math.cos(YAW)*math.cos(PITCH);
    T[0,1] = math.cos(YAW)*math.sin(PITCH)*math.sin(ROLL)-math.sin(YAW)*math.cos(ROLL);
    T[0,2] = math.cos(YAW)*math.sin(PITCH)*math.cos(ROLL)+math.sin(YAW)*math.sin(ROLL);
    T[0,3] = X;
    T[1,0] = math.sin(YAW)*math.cos(PITCH);
    T[1,1] = math.sin(YAW)*math.sin(PITCH)*math.sin(ROLL)+math.cos(YAW)*math.cos(ROLL);
    T[1,2] = math.sin(YAW)*math.sin(PITCH)*math.cos(ROLL)-math.cos(YAW)*math.sin(ROLL);
    T[1,3] = Y;
    T[2,0] = -math.sin(PITCH);
    T[2,1] = math.cos(PITCH)*math.sin(ROLL);
    T[2,2] = math.cos(PITCH)*math.cos(ROLL);
    T[2,3] = Z;
    T[3,3] = 1;
    return T

## Configuration parameters.

In [6]:
# .stl file path
fileName = "../../PLuM/sample_data/FLATFOOT_StanfordBunny_jmil_HIGH_RES_Smoothed.stl"
outputFileName = 'bunny_lookup_table_sigma_1_05_mm.lookup'
lookupToModel = homogeneous(0,0,0,50,40,10)
stepSize = 0.5
maxXYZ = [100,100,100] # [x,y,z]
sigma = 1

## Read the mesh file.

In [7]:
## load mesh and convert to open3d.t.geometry.TriangleMesh
mesh = o3d.io.read_triangle_mesh(fileName)
mesh = o3d.t.geometry.TriangleMesh.from_legacy(mesh)

# minimum and maximum geometry bounds
min_bound = mesh.vertex.positions.min(0).numpy()
max_bound = mesh.vertex.positions.max(0).numpy()
print(min_bound)
print(max_bound)

## transform the mesh to the correct frame
mesh_t = copy.deepcopy(mesh).transform(lookupToModel)

## Create a scene and add the triangle mesh
scene = o3d.t.geometry.RaycastingScene()
_ = scene.add_triangles(mesh_t)  # we do not need the geometry ID for mesh

## updated minimum and maximum geometry bounds 
min_bound = mesh_t.vertex.positions.min(0).numpy()
max_bound = mesh_t.vertex.positions.max(0).numpy()
print(min_bound)
print(max_bound)

[-43.148468 -33.433746   0.      ]
[43.148468 33.433746 83.74813 ]
[ 6.851532   6.5662537 10.       ]
[93.14847 73.43375 93.74813]


In [8]:
query_point = o3d.core.Tensor([[0,0,0]], dtype=o3d.core.Dtype.Float32)
            # print(query_point.numpy())
unsigned_distance = scene.compute_distance(query_point)
print(unsigned_distance.numpy())

[30.418768]


## Generate the Lookup table.

In [ ]:
closestDistance = []
closestOccupancy = []
rewardValues = []

numX = round(maxXYZ[0]/stepSize) + 1
numY = round(maxXYZ[1]/stepSize) + 1
numZ = round(maxXYZ[2]/stepSize) + 1

xDim = np.linspace(0,maxXYZ[0],numX)
yDim = np.linspace(0,maxXYZ[1],numY)
zDim = np.linspace(0,maxXYZ[2],numZ)

f = open(outputFileName, 'w+b')

for x in xDim:
    for y in yDim:
        for z in zDim:
            query_point = o3d.core.Tensor([[x, y, z]], dtype=o3d.core.Dtype.Float32)
            # print(query_point.numpy())
            unsigned_distance = scene.compute_distance(query_point)
            occupancy = scene.compute_occupancy(query_point)
            reward = int(255 * math.exp(-0.5 * unsigned_distance.numpy() * unsigned_distance.numpy() / (sigma * sigma)));
            closestDistance.append(unsigned_distance.numpy())
            closestOccupancy.append(occupancy.numpy())
            rewardValues.append(reward)
            
            binary_format = bytearray([reward])
            f.write(binary_format)

f.close()

# rewardsToWrite = np.array(rewardValues,dtype=np.uint8)
closestDistancesToWrite = np.array(closestDistance)
occupancyToWrite = np.array(closestOccupancy)
np.savetxt('lookupTable.txt', closestDistancesToWrite, delimiter=',')
np.savetxt('lookupTableOccupancy.txt', occupancyToWrite, delimiter=',')
# with open('test1.txt', 'w') as f:
#     for pt in closestDistance:
#         f.write(f"{pt}\n")